# Sketchmap Demo

In [ ]:
import os
import os.path
import re
import time
import threading
import subprocess

import ipywidgets as widgets
from IPython.display import display

In [ ]:
glosim_fn = "glosim.out"
glosim_process = None

def glosim_output_worker():
    pre_tag = '<pre style="width:600px; max-height:250px; overflow-x:auto; line-height:1em; font-size:0.8em;">'   
    while(True):
        if os.path.exists(glosim_fn):
            output = open(glosim_fn).read() # TODO seek forward
            output = output.strip("\r")
            output = re.sub("\n.*\r", "\n", output)
            output_area.value = pre_tag + output + '</pre>'
        if(glosim_process.poll() != None):
            break
        time.sleep(1)
    btn_startstop.description="Start glosim"

def start_glosim():
    global glosim_process
    btn_startstop.description="Stop glosim"
    
    if os.path.exists(glosim_fn):
        os.remove(glosim_fn)
    logfile = open(glosim_fn, "w")
    cmd = "glosim.py traj.lys.dipeptide.bare.xyz -n 10 -l 8 -c 3.5 --kernel fastavg --distance --periodic --nonorm --np 2 --nlandmarks 300"
    glosim_process = subprocess.Popen(cmd.split(), stdout=logfile, stderr=logfile)
    
    # start output thread
    threading.Thread(target=glosim_output_worker).start() 

In [ ]:
def on_click_startstop(e):
    if(btn_startstop.description.startswith("Start glosim")):
        start_glosim()
    else:
        glosim_process.kill()
     
btn_startstop = widgets.Button(description="Start glosim")
btn_startstop.on_click(on_click_startstop)
output_area = widgets.HTML()

display(widgets.VBox([btn_startstop, output_area]))